In [37]:
import lmppl
import pandas as pd
import time
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm
scorer = lmppl.OpenAI(api_key="", model="ada", sleep_time=0)

In [ ]:
path = "./low_res/{}/lexglue/eda/{}_train.tsv"

dataset_to_col_name_mapping = {
    "ledgar": "text",
    "scotus": "text",
    "ildc": "text",
    "ots": "text",
    "ots_topics": "0",
    "case_hold": "context",
    "ecthr_a": "0",
    "ecthr_b": "0",
    "eurlex": "0",
    "unfair_tos": "0"
}

def truncate_1500(text_list):
    final_list = []
    for text in (text_list):
        sentences = sent_tokenize(text)
        final_sent = ''
        for sent in sentences:
            if len(final_sent + sent) <= 1500:
                final_sent = final_sent + sent
            else:
                break
        if (len(final_sent)==0):
            final_sent = sentences[0][:1500]
        final_list.append(final_sent)
    
    return final_list

ppl_list = []

# for split in [500]:
for split in [200, 500, 1000]:
    for dataset in ["ledgar", "scotus", "ecthr_a", "ecthr_b", "eurlex", "case_hold"]:
    # for dataset in ["ecthr_b"]:
        try:
            actual_path = path.format(split, dataset)
            df = pd.read_csv(actual_path, sep="\t", header=0)
            text = df[dataset_to_col_name_mapping[dataset]]
            text = truncate_1500(text)
            ppl = scorer.get_perplexity(text)
            print(f"For split : {split}, the avg score for dataset : {dataset} is {sum(ppl)/len(ppl)}")
            ppl_list.append(ppl)
            time.sleep(10)
        except Exception as ex:
            print(f"For split {split}, dataset {dataset} encountered exception : {ex}")